In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# Check the folder you are in
%ls

gdrive/  sample_data/


In [3]:
# Run this in case you are not in the right folder
%cd gdrive
%cd My Drive
%cd PatternRecognitionProject2020

/content/gdrive
/content/gdrive/My Drive
/content/gdrive/My Drive/PatternRecognitionProject2020


In [4]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 26 19:14:36 2018

@author: omdiv
"""



#without training embedding 
#Mean Squared Error = 0.0309009
#accuracy = 0.857043719639
#precision_score = 0.778106508876
#recall_score = 0.568443804035
#f1_score = 0.65695253955



from __future__ import print_function
from sklearn.model_selection import KFold
import numpy as np
import os
from sklearn import metrics
import json
import pickle
from collections import Counter
np.random.seed(42)
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Bidirectional
from keras.layers import LSTM, SimpleRNN, GRU,Flatten,RepeatVector,Permute,Conv1D,GlobalMaxPooling1D
from keras.models import load_model
import keras.callbacks
import re
import nltk
# from tweet_utils import *

PAD = "<pad>"  
UNK = "<unk>"  
nltk_tokeniser = nltk.tokenize.TweetTokenizer()

EmbeddingSize = 100


def WordEmbeddingLoader(fp, embedding_size):
    embedding = []
    vocab = []
    linenumber=0
    with open(fp, 'r', encoding='UTF-8') as f:
        for each_line in f:
          
            linenumber+=1            
            row = each_line.split(' ')
 
            if len(row) == 2:
                continue
            vocab.append(row[0])
            if len(row[1:]) != embedding_size:
                print (row[0])
                print (len(row[1:]))
            embedding.append(np.asarray(row[1:], dtype='float32'))
   
    word2id = dict(zip(vocab, range(2, len(vocab)+2)))
    word2id[PAD] = 0
    word2id[UNK] = 1

    extra_embedding = [np.zeros(embedding_size), np.random.uniform(-0.1, 0.1, embedding_size)]
    embedding = np.append(extra_embedding, embedding, 0)
    return word2id, embedding,vocab

def data_reader(fps, word2id=None, y_len=1, use_target_description=False, use_image=False, delete_irregularities=False):
    ids = []
    post_texts = []
    post_text_lens = []
    truth_means = []
    truth_classes = []
    id2truth_class = {}
    id2truth_mean = {}
    target_descriptions = []
    target_description_lens = []
    image_features = []
    num = 0
    for fp in fps:
        if use_image:
            with open(os.path.join(fp, "id2imageidx.json"), "r") as fin:
                id2imageidx = json.load(fin)
            
            all_image_features = pickle.load(os.path.join(fp, "image_features.hkl"))
        if y_len:
            with open(os.path.join(fp, 'truth.jsonl'), 'rb') as fin:
                for each_line in fin:
                    each_item = json.loads(each_line.decode('utf-8'))
                    if delete_irregularities:
                        if each_item["truthClass"] == "clickbait" and float(each_item["truthMean"]) < 0.5 or each_item["truthClass"] != "clickbait" and float(each_item["truthMean"]) > 0.5:
                            continue
                    if y_len == 4:
                        each_label = [0, 0, 0, 0]
                        
                        for each_key, each_value in Counter(each_item["truthJudgments"]).items():
                            each_label[int(each_key//0.3)] = float(each_value)/5
                        id2truth_class[each_item["id"]] = each_label
                        if each_item["truthClass"] != "clickbait":
                            assert each_label[0]+each_label[1] > each_label[2]+each_label[3]
                        else:
                            assert each_label[0]+each_label[1] < each_label[2]+each_label[3]
                    if y_len == 2:
                        if each_item["truthClass"] == "clickbait":
                            id2truth_class[each_item["id"]] = [1, 0]
                        else:
                            id2truth_class[each_item["id"]] = [0, 1]
                    if y_len == 1:
                        if each_item["truthClass"] == "clickbait":
                            id2truth_class[each_item["id"]] = [1]
                        else:
                            id2truth_class[each_item["id"]] = [0]
                    id2truth_mean[each_item["id"]] = [float(each_item["truthMean"])]
        
        with open(os.path.join(fp, 'instances.jsonl'), 'rb') as fin:
            for each_line in fin:
                each_item = json.loads(each_line.decode('utf-8'))
                if each_item["id"] not in id2truth_class and y_len:
                    num += 1
                    continue
                ids.append(each_item["id"])
                each_post_text = " ".join(each_item["postText"])
                each_target_description = each_item["targetTitle"]
                if y_len:
                    truth_means.append(id2truth_mean[each_item["id"]])
                    truth_classes.append(id2truth_class[each_item["id"]])
                if word2id:
                    if (each_post_text+" ").isspace():
                        #the id of <unk>
                        post_texts.append([0])
                        post_text_lens.append(1)
                    else:
                        each_post_tokens = tokeniser(each_post_text)
                        post_texts.append([word2id.get(each_token, 1) for each_token in each_post_tokens])
                        post_text_lens.append(len(each_post_tokens))
                else:
                    post_texts.append([each_post_text])
                if use_target_description:
                    if word2id:
                        if (each_target_description+" ").isspace():
                            target_descriptions.append([0])
                            target_description_lens.append(1)
                        else:
                            each_target_description_tokens = tokeniser(each_target_description)
                            target_descriptions.append([word2id.get(each_token, 1) for each_token in each_target_description_tokens])
                            target_description_lens.append(len(each_target_description_tokens))
                    else:
                        target_descriptions.append([each_target_description])
                else:
                    target_descriptions.append([])
                    target_description_lens.append(0)
                if use_image:
                    image_features.append(all_image_features[id2imageidx[each_item["id"]]].flatten())
                else:
                    image_features.append([])
    print ("Deleted number of items: " + str(num))
    return ids, post_texts, truth_classes, post_text_lens, truth_means, target_descriptions, target_description_lens, image_features



def Sequence_pader(sequences, maxlen):
    if maxlen <= 0:
        return sequences
    shape = (len(sequences), maxlen)
    padded_sequences = np.full(shape, 0)
    for i, each_sequence in enumerate(sequences):
        if len(each_sequence) > maxlen:
            padded_sequences[i] = each_sequence[:maxlen]
        else:
            padded_sequences[i, :len(each_sequence)] = each_sequence
    return padded_sequences



def tweet_tokenizer(text):
    return simpleTokenize(squeezeWhitespace(text))

def tokeniser(text, with_process=True):
    if with_process:
        return nltk_tokeniser.tokenize(tweet_processor(text).lower())
    else:
        # return nltk_tokeniser.tokenize(text)
        return tweet_tokenizer(text.lower())




def tweet_processor(text):
    FLAGS = re.MULTILINE | re.DOTALL
    
    def megasplit(pattern, string):
        splits = list((m.start(), m.end()) for m in re.finditer(pattern, string))
        starts = [0] + [i[1] for i in splits]
        ends = [i[0] for i in splits] + [len(string)]
        return [string[start:end] for start, end in zip(starts, ends)]
    
    def hashtag(text):
        text = text.group()
        hashtag_body = text[1:]
        #print(hashtag_body)
        
        #result = " ".join(["<hashtag>"] + re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS))
        result = " ".join(["<hashtag>"] + megasplit(r"(?=[A-Z])", hashtag_body))
        return result

    def allcaps(text):
        text = text.group()
        return text.lower() + " <allcaps>"
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")

    ## -- I just don't understand why the Ruby script adds <allcaps> to everything so I limited the selection.
    # text = re_sub(r"([^a-z0-9()<>'`\-]){2,}", allcaps)
    text = re_sub(r"([A-Z]){2,}", allcaps)

    return text



np.random.seed(81)
word2id, embedding_matrix,vocab = WordEmbeddingLoader(fp=os.path.join('data', "glove.6B."+str(EmbeddingSize)+"d.txt"), embedding_size=EmbeddingSize)
with open(os.path.join('data', 'word2id.json'), 'w') as fout:
    json.dump(word2id, fp=fout)
    

ids, post_texts, truth_classes, post_text_lens, truth_means, target_descriptions, target_description_lens, image_features = data_reader(word2id=word2id, fps=[os.path.join('data', 'clickbait17-validation'), os.path.join('new_data', 'clickbait17-train-170331')], y_len=4, use_target_description=False, use_image=False)
print(type(post_texts))
post_texts = np.array(post_texts)

print("#######################")
print(post_texts.shape)
truth_classes = np.array(truth_classes)
post_text_lens = np.array(post_text_lens)
truth_means = np.array(truth_means)
shuffle_indices = np.random.permutation(np.arange(len(post_texts)))
post_texts = post_texts[shuffle_indices]
truth_classes = truth_classes[shuffle_indices]
post_text_lens = post_text_lens[shuffle_indices]
truth_means = truth_means[shuffle_indices]
print(post_text_lens)
max_post_text_len = max(post_text_lens)


post_texts = Sequence_pader(post_texts, max_post_text_len)

target_descriptions = np.array(target_descriptions)
target_description_lens = np.array(target_description_lens)
target_descriptions = target_descriptions[shuffle_indices]
target_description_lens = target_description_lens[shuffle_indices]
max_target_description_len = max(target_description_lens)
print(max_target_description_len)
target_descriptions = Sequence_pader(target_descriptions, max_target_description_len)




tetids, tepost_texts, tetruth_classes, tepost_text_lens, tetruth_means, tetarget_descriptions, tetarget_description_lens, teimage_features = data_reader(word2id=word2id, fps=[os.path.join('data', 'clickbait17-test')], y_len=4, use_target_description=False, use_image=False)    
tepost_texts = np.array(tepost_texts)
tetruth_classes = np.array(tetruth_classes)
tepost_text_lens = [each_len if each_len <= max_post_text_len else max_post_text_len for each_len in tepost_text_lens]
tepost_text_lens = np.array(tepost_text_lens)
tetruth_means = np.array(tetruth_means)
tetruth_means = np.ravel(tetruth_means).astype(np.float32)
tepost_texts = Sequence_pader(tepost_texts, max_post_text_len)


max_features = len(word2id.keys())
print(max_features)
maxlen = max_post_text_len
embedding_dims = EmbeddingSize
dropout_embedding = 0.2
X_train = post_texts
y_train = truth_means
print("X_train: ", X_train.shape)
print(X_train)
print(y_train)

X_test = tepost_texts
y_test = tetruth_means

print("X_test: ", X_test.shape)
print(X_test)
print(y_test)


# build the keras LSTM model
model = Sequential()

model.add(Embedding(input_dim = max_features,
                    output_dim = embedding_dims,
                    weights = [embedding_matrix],
                    input_length = maxlen
                    ,trainable = False))
model.add(Dropout(dropout_embedding))

model.add(Bidirectional(GRU(256, dropout_W=0.2, dropout_U=0.5)))  
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='mse',
              optimizer='rmsprop')

batch_size = 64

earlystop_cb = keras.callbacks.EarlyStopping(monitor='mse', patience=7, verbose=1, mode='auto')

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=2,
          validation_split=0.1, callbacks=[earlystop_cb])

# # # save the model
# # model.save('model_albacore.h5')

# # # only save the weights
# # model.save_weights('weights_albacore.hdf5')

petruth_means = model.predict(X_test)
tetruthClass = []
petruthClass = []

hi_count = 0
lo_count = 0

for i in range(len(tetruth_means)):
  
    if petruth_means[i] > 0.5:
        petruthClass.append(1)
    else:
        petruthClass.append(0)
    
    if tetruth_means[i] > 0.5:
        tetruthClass.append(1)
        hi_count += 1
    else:
        tetruthClass.append(0)
        lo_count += 1

print(hi_count, lo_count)


mse = metrics.mean_squared_error(tetruth_means,petruth_means)
print('Mean Squared Error = '+str(mse))

accuracy = metrics.accuracy_score(tetruthClass,petruthClass)
print('accuracy = '+str(accuracy))

precision = metrics.precision_score(tetruthClass,petruthClass)
print('precision_score = '+str(precision))

recall = metrics.recall_score(tetruthClass,petruthClass)
print('recall_score = '+str(recall))

f1 = metrics.f1_score(tetruthClass,petruthClass)
print('f1_score = '+str(f1))

conf = metrics.confusion_matrix(tetruthClass,petruthClass)
print(conf)

Using TensorFlow backend.


Deleted number of items: 0
<class 'list'>
#######################
(16135,)
[15 10 17 ... 10 17 16]
0
Deleted number of items: 0
400002
X_train:  (16135, 42)
[[   92     1  2254 ...     0     0     0]
 [ 1033 10785 15081 ...     0     0     0]
 [    1  1611  1244 ...     0     0     0]
 ...
 [ 3540   909   501 ...     0     0     0]
 [18340 50435     3 ...     0     0     0]
 [    2     1  4117 ...     0     0     0]]
[[0.2       ]
 [0.13333334]
 [0.26666667]
 ...
 [0.13333333]
 [0.06666667]
 [0.53333333]]
X_test:  (5862, 42)
[[  39   16  221 ...    0    0    0]
 [   2  222    1 ...    0    0    0]
 [ 481    6  348 ...    0    0    0]
 ...
 [ 159 3236    3 ...    0    0    0]
 [1344 1735  147 ...    0    0    0]
 [   1    1    1 ...    0    0    0]]
[1.         0.73333335 0.6        ... 0.2        0.4        0.8       ]










Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:316: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(256, dropout=0.2, recurrent_dropout=0.5)`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:328: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 14521 samples, validate on 1614 samples
Epoch 1/2
14521/14521 [==============================] - 34s 2ms/step - loss: 0.0460 - val_loss: 0.0343
Epoch 2/2
   64/14521 [..............................] - ETA: 31s - loss: 0.0396

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `mse` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


14521/14521 [==============================] - 31s 2ms/step - loss: 0.0391 - val_loss: 0.0322
1415 4447
Mean Squared Error = 0.034079634
accuracy = 0.8369157284203343
precision_score = 0.705829596412556
recall_score = 0.5561837455830388
f1_score = 0.6221343873517785
[[4119  328]
 [ 628  787]]


In [5]:
import json
import ast
import pandas as pd

# get dict with article ids (key) and image ids (value)
def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

# dataType is "train" or "test"
def getArticles(dataType):
  # get dict with image ids (key) and captions (value)
  imgIdToCaption = dict()
  def generateImgIdToCaption(type):
    fp_image = "new_data/caption-" + type
    with open(fp_image, 'r', encoding='UTF-8') as file:
      content = file.read()
      content = ast.literal_eval(content)
      for key in content:
        value = content[key]
        if isinstance(value, list):
          value = value[0]
        imgIdToCaption[key] = value
  
  if dataType == "train":
    # use both train and validation as train (merged)
    print(len(imgIdToCaption.keys()))
    generateImgIdToCaption("validation")
    print(len(imgIdToCaption.keys()))

    generateImgIdToCaption("train")
    print(len(imgIdToCaption.keys()))
  elif dataType == "test":
    generateImgIdToCaption("test")
  
  print("########################")
  print(len(imgIdToCaption.keys()))

  # load the clickbait truth labels
  articles = []
  truths = []
  if dataType == "train":
      truths += load_jsonl("new_data/clickbait17-train-170331/truth.jsonl")
      truths += load_jsonl("new_data/clickbait17-validation/truth.jsonl")
      articles += load_jsonl("new_data/clickbait17-train-170331/instances.jsonl")
      articles += load_jsonl("new_data/clickbait17-validation/instances.jsonl")
  elif dataType == "test":
    truths += load_jsonl("new_data/clickbait17-test/truth.jsonl")
    articles += load_jsonl("new_data/clickbait17-test/instances.jsonl")
  else:
    print("OOPS")
  print(len(truths), len(articles))

  # create lists of postTexts, imageCaptions and clickbaitScores
  postTexts = []
  imageCaptions = []
  clickbaitScores = []
  counter = 0
  nonmatchSkip = 0
  noMediaOrTextSkip = 0
  for article in articles:
    postText = article["postText"][0]
    if len(article["postMedia"]) > 0 and len(postText) > 0: # only consider articles with at least one image and a post title
      if article["id"] in imgIdToCaption.keys():
        imageID = article["id"]

        # find matching id in truth and assign turhtMean
        for truth in truths:
          if truth["id"] == imageID:
            clickbaitScores.append(truth["truthMean"])

        # step 2: post text
        postTexts.append(postText)
        imageCaptions.append(imgIdToCaption[imageID])
    
        # step 3: clickbait score (truth)
        # if truths[counter]['id'] != imageID:
        #   print(truths[counter]['id'])
        #   print(imageID)
        #   print("Hier gaat het mis met de counter")
        # print(truths[1])
        # print(truths[].index(imageID))

        # clickbaitScore = truths[truths.index(imageID)]['truthMean']

        # clickbaitScores.append([clickbaitScore])

      else:
        nonmatchSkip += 1
    else:
      noMediaOrTextSkip += 1
    counter += 1
  print("Skipped " + str(nonmatchSkip) + " due to non matches")
  print("Skipped " + str(noMediaOrTextSkip) + " due to articles not having an image or post text")
  print(len(postTexts))
  return postTexts, imageCaptions, clickbaitScores

postTexts_train, imageCaptions_train, clickbaitScores_train = getArticles("train")
postTexts_test, imageCaptions_test, clickbaitScores_test = getArticles("test")
# postTexts_validation, imageCaptions_validation, clickbaitScores_validation = getArticles("validation")
# print("postTexts, imageCaptions and clickbaitScores of train folder: ", getArticles("train"))
# print("postTexts, imageCaptions and clickbaitScores of test folder: ", getArticles("test"))
# print("postTexts, imageCaptions and clickbaitScores of validation folder: ", getArticles("validation"))

print(len(imageCaptions_train))
print(len(clickbaitScores_train))


0
7101
8724
########################
8724
Loaded 2459 records from new_data/clickbait17-train-170331/truth.jsonl
Loaded 13676 records from new_data/clickbait17-validation/truth.jsonl
Loaded 2459 records from new_data/clickbait17-train-170331/instances.jsonl
Loaded 13676 records from new_data/clickbait17-validation/instances.jsonl
16135 16135
Skipped 129 due to non matches
Skipped 7282 due to articles not having an image or post text
8724
########################
3031
Loaded 5862 records from new_data/clickbait17-test/truth.jsonl
Loaded 5862 records from new_data/clickbait17-test/instances.jsonl
5862 5862
Skipped 45 due to non matches
Skipped 2786 due to articles not having an image or post text
3031
8724
8724


In [6]:
from __future__ import print_function
from sklearn.model_selection import KFold
import numpy as np
import os
from sklearn import metrics
import json
import pickle
from collections import Counter
np.random.seed(42)
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Bidirectional
from keras.layers import LSTM, SimpleRNN, GRU,Flatten,RepeatVector,Permute,Conv1D,GlobalMaxPooling1D
from keras.models import load_model
import keras.callbacks
import re
import nltk
from tweet_utils import *

PAD = "<pad>"  
UNK = "<unk>"  
nltk_tokeniser = nltk.tokenize.TweetTokenizer()

EmbeddingSize = 100

def WordEmbeddingLoader(fp, embedding_size):
    embedding = []
    vocab = []
    linenumber=0
    with open(fp, 'r', encoding='UTF-8') as f:
        for each_line in f:
          
            linenumber+=1            
            row = each_line.split(' ')
 
            if len(row) == 2:
                continue
            vocab.append(row[0])
            if len(row[1:]) != embedding_size:
                print (row[0])
                print (len(row[1:]))
            embedding.append(np.asarray(row[1:], dtype='float32'))
   
    word2id = dict(zip(vocab, range(2, len(vocab)+2)))
    word2id[PAD] = 0
    word2id[UNK] = 1

    extra_embedding = [np.zeros(embedding_size), np.random.uniform(-0.1, 0.1, embedding_size)]
    embedding = np.append(extra_embedding, embedding, 0)
    return word2id, embedding,vocab


def Sequence_pader(sequences, maxlen):
    if maxlen <= 0:
        return sequences
    shape = (len(sequences), maxlen)
    padded_sequences = np.full(shape, 0)
    for i, each_sequence in enumerate(sequences):
        if len(each_sequence) > maxlen:
            padded_sequences[i] = each_sequence[:maxlen]
        else:
            padded_sequences[i, :len(each_sequence)] = each_sequence
    return padded_sequences



def tweet_tokenizer(text):
    return simpleTokenize(squeezeWhitespace(text))

def tokeniser(text, with_process=True):
    if with_process:
        return nltk_tokeniser.tokenize(tweet_processor(text).lower())
    else:
        # return nltk_tokeniser.tokenize(text)
        return tweet_tokenizer(text.lower())



def tweet_processor(text):
    FLAGS = re.MULTILINE | re.DOTALL
    
    def megasplit(pattern, string):
        splits = list((m.start(), m.end()) for m in re.finditer(pattern, string))
        starts = [0] + [i[1] for i in splits]
        ends = [i[0] for i in splits] + [len(string)]
        return [string[start:end] for start, end in zip(starts, ends)]
    
    def hashtag(text):
        text = text.group()
        hashtag_body = text[1:]
        #print(hashtag_body)
        
        #result = " ".join(["<hashtag>"] + re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS))
        result = " ".join(["<hashtag>"] + megasplit(r"(?=[A-Z])", hashtag_body))
        return result

    def allcaps(text):
        text = text.group()
        return text.lower() + " <allcaps>"
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")

    ## -- I just don't understand why the Ruby script adds <allcaps> to everything so I limited the selection.
    # text = re_sub(r"([^a-z0-9()<>'`\-]){2,}", allcaps)
    text = re_sub(r"([A-Z]){2,}", allcaps)

    return text

np.random.seed(81)
word2id, embedding_matrix,vocab = WordEmbeddingLoader(fp=os.path.join('data', "glove.6B."+str(EmbeddingSize)+"d.txt"), embedding_size=EmbeddingSize)
with open(os.path.join('data', 'word2id.json'), 'w') as fout:
    json.dump(word2id, fp=fout)


def embedding(to_embed):
  embedding = []
  lens = []
  for text in to_embed:
    if (text+" ").isspace():
      embedding.append([0])
      lens.append(1)
    else:
      each_text_tokens = tokeniser(text)
      embedding.append([word2id.get(each_token, 1) for each_token in each_text_tokens])
      lens.append(len(each_text_tokens))
  return embedding,lens

def Sequence_pader(sequences, maxlen):
    if maxlen <= 0:
        return sequences
    shape = (len(sequences), maxlen)
    padded_sequences = np.full(shape, 0)
    for i, each_sequence in enumerate(sequences):
        if len(each_sequence) > maxlen:
            padded_sequences[i] = each_sequence[:maxlen]
        else:
            padded_sequences[i, :len(each_sequence)] = each_sequence
    return padded_sequences

## clickbait
clickbaitScores_train = np.array(clickbaitScores_train)
clickbaitScores_test = np.array(clickbaitScores_test)
clickbaitScores_test = np.ravel(clickbaitScores_test).astype(np.float32)
# clickbaitScores_validation = np.array(clickbaitScores_validation)

## postText
#embeddings
print(postTexts_train)
postTexts_train_emb, postTexts_train_len = embedding(postTexts_train)
print(type(postTexts_train_emb))
postTexts_test_emb, postTexts_test_len = embedding(postTexts_test)
# postTexts_validation_emb, postTexts_validation_len = embedding(postTexts_validation)

#np arrays
postTexts_train_emb = np.array(postTexts_train_emb)
postTexts_train_len = np.array(postTexts_train_len)
max_postTexts_train_len = max(postTexts_train_len)

postTexts_test_emb = np.array(postTexts_test_emb)
postTexts_test_len = np.array(postTexts_test_len)
max_postTexts_test_len = max(postTexts_test_len)

# postTexts_validation_emb = np.array(postTexts_validation_emb)
# postTexts_validation_len = np.array(postTexts_validation_len)
# max_postTexts_validation_len = max(postTexts_validation_len)

maxlen_posText = max(max_postTexts_train_len, max_postTexts_test_len)


#padding
postTexts_train_emb = Sequence_pader(postTexts_train_emb, maxlen_posText)
postTexts_test_emb = Sequence_pader(postTexts_test_emb, maxlen_posText)
# postTexts_validation_emb = Sequence_pader(postTexts_validation_emb, maxlen_posText)


## Captions
#embeddings
imageCaptions_train_emb, imageCaptions_train_len = embedding(imageCaptions_train)
imageCaptions_test_emb, imageCaptions_test_len = embedding(imageCaptions_test)
# imageCaptions_validation_emb, imageCaptions_validation_len = embedding(imageCaptions_validation)

#np arrays
imageCaptions_train_emb = np.array(imageCaptions_train_emb)
imageCaptions_train_len = np.array(imageCaptions_train_len)
max_imageCaptions_train_len = max(imageCaptions_train_len)


imageCaptions_test_emb = np.array(imageCaptions_test_emb)
imageCaptions_test_len = np.array(imageCaptions_test_len)
max_imageCaptions_test_len = max(imageCaptions_test_len)

# imageCaptions_validation_emb = np.array(imageCaptions_validation_emb)
# imageCaptions_validation_len = np.array(imageCaptions_validation_len)
# max_imageCaptions_validation_len = max(imageCaptions_validation_len)

maxlen_captions = max(max_imageCaptions_train_len, max_imageCaptions_test_len)
# print(max_imageCaptions_train_len, max_imageCaptions_test_len)
print(maxlen_captions)


#padding
imageCaptions_train_emb = Sequence_pader(imageCaptions_train_emb, maxlen_captions)
imageCaptions_test_emb = Sequence_pader(imageCaptions_test_emb, maxlen_captions)
# imageCaptions_validation_emb = Sequence_pader(imageCaptions_validation_emb, maxlen_captions)

print(imageCaptions_train_emb.shape)
print(imageCaptions_test_emb.shape)
# print(imageCaptions_validation_emb.shape)


# print(imageCaptions_validation_emb)
# print(clickbaitScores_validation)


# shuffle
shuffle_indices = np.random.permutation(np.arange(len(clickbaitScores_train)))
clickbaitScores_train = clickbaitScores_train[shuffle_indices]
postTexts_train_emb = postTexts_train_emb[shuffle_indices]
imageCaptions_train_emb = imageCaptions_train_emb[shuffle_indices]



['RT @kenbrown12: Emerging market investors are doing their best Monty Pythons--"Run away, run away"', 'Could light bulbs hurt your health? One company is now putting warning labels on its bulbs:', '13 classic ’00s songs that were actually meant for other artists', 'Dez Bryant is reportedly considering skipping Week 1 game if he doesn’t get a long-term deal', 'House #GOP plans two days of debate, Friday showdown vote on Obama trade agenda.', 'Only one in three of us complain when we are unhappy with the NHS...', 'Britain forced to withdraw spies after Russia and China access files stolen by Snowden', 'Tourists detained in Malaysia for getting naked on sacred mountain:', "Chinese women are growing their armpit hair like they just don't care", 'RT @BBCWalesNews: Caerphilly farmer may get payout worth hundreds of thousands after 24 year wait', 'Obama actually writes back to people who call him an idiot', "RT @TheFix: The president's 'bully pulpit' is way overrated. This trade vote proved 

In [7]:
## Take equal subsets 

#  max = min(len(np.where(clickbaitScores_train > 0.5)[1]), len(np.where(clickbaitScores_train < 0.5)[1]))
treshold = 0.5
indeces1 = np.where(clickbaitScores_train > treshold)[0]
indeces2 = np.where(clickbaitScores_train <= treshold)[0]

# scores
clickbaitScores_train_eqsub1 = clickbaitScores_train[indeces1]
clickbaitScores_train_eqsub2 = clickbaitScores_train[indeces2[0:len(indeces1)]]
print(type(clickbaitScores_train_eqsub1))
clickbaitScores_train_eqsub =  np.concatenate((clickbaitScores_train_eqsub1, clickbaitScores_train_eqsub2))


# postTexts
postTexts_train_emb_eqsub1 = postTexts_train_emb[indeces1]
postTexts_train_emb_eqsub2 = postTexts_train_emb[indeces2[0:len(indeces1)]]
postTexts_train_emb_eqsub =  np.concatenate((postTexts_train_emb_eqsub1, postTexts_train_emb_eqsub2))

# captions
imageCaptions_train_emb_eqsub1 = imageCaptions_train_emb[indeces1]
imageCaptions_train_emb_eqsub2 = imageCaptions_train_emb[indeces2[0:len(indeces1)]]
imageCaptions_train_emb_eqsub =  np.concatenate((imageCaptions_train_emb_eqsub1, imageCaptions_train_emb_eqsub2))

print(imageCaptions_train_emb_eqsub)
print(len(imageCaptions_train_emb_eqsub))


<class 'numpy.ndarray'>
[[   9  789    8 ...    0    0    0]
 [   9  131    5 ...    0    0    0]
 [   9  789 2997 ...    0    0    0]
 ...
 [   9 1336    5 ...    0    0    0]
 [   9  302    8 ...    0    0    0]
 [   9  131    5 ...    0    0    0]]
3978


In [8]:
print("XXXXXXXXXXXX OWN")
print(postTexts_train_emb_eqsub)
print(clickbaitScores_train_eqsub)


print(postTexts_train_emb_eqsub.shape)
print(clickbaitScores_train_eqsub.shape)
# print(postTexts_train_emb_eqsub)
# print(clickbaitScores_test)


print("XXXXXXXXX TEST & TRAIN")
print(X_train)
print(y_train)
print(X_train.shape)
print(y_train.shape)


# print(embedding)

XXXXXXXXXXXX OWN
[[    1  7236  2672 ...     0     0     0]
 [  740    34   102 ...     0     0     0]
 [33224     1     1 ...     0     0     0]
 ...
 [   71    34   624 ...     0     0     0]
 [    2   216     5 ...     0     0     0]
 [ 2672    47     9 ...     0     0     0]]
[0.53333333 0.8        0.53333336 ... 0.4        0.26666667 0.33333333]
(3978, 39)
(3978,)
XXXXXXXXX TEST & TRAIN
[[   92     1  2254 ...     0     0     0]
 [ 1033 10785 15081 ...     0     0     0]
 [    1  1611  1244 ...     0     0     0]
 ...
 [ 3540   909   501 ...     0     0     0]
 [18340 50435     3 ...     0     0     0]
 [    2     1  4117 ...     0     0     0]]
[[0.2       ]
 [0.13333334]
 [0.26666667]
 ...
 [0.13333333]
 [0.06666667]
 [0.53333333]]
(16135, 42)
(16135, 1)


In [9]:
# train model captions
max_features = len(word2id.keys())
embedding_dims = EmbeddingSize
dropout_embedding = 0.2
maxlen = maxlen_captions

# build the keras LSTM model
model_captions = Sequential()

model_captions.add(Embedding(input_dim = max_features,
                    output_dim = embedding_dims,
                    weights = [embedding_matrix],
                    input_length = maxlen
                    ,trainable = False))
model_captions.add(Dropout(dropout_embedding))

model_captions.add(Bidirectional(GRU(32, dropout_W=0.2, dropout_U=0.1)))  
model_captions.add(Dense(1))
model_captions.add(Activation('sigmoid'))
# model_captions.add(Dense(1))
# model_captions.add(Activation('sigmoid'))

model_captions.compile(loss='mse',
              optimizer='rmsprop')

batch_size = 32

earlystop_cb = keras.callbacks.EarlyStopping(monitor='mse', patience=7, verbose=1, mode='auto')

model_captions.fit(imageCaptions_train_emb_eqsub, clickbaitScores_train_eqsub, batch_size=batch_size, nb_epoch=2,
          validation_split=0.1, callbacks=[earlystop_cb])

# save the model
# model_captions.save('model_captions.h5')

# only save the weights
# model_captions.save_weights('weights_captions.hdf5')

petruth_means = model_captions.predict(imageCaptions_test_emb)
print(petruth_means)
tetruthClass = []
petruthClass = []

count_high = 0
count_low = 0
for i in range(len(clickbaitScores_test)):
  
    if petruth_means[i] > treshold:
        petruthClass.append(1)
    else:
        petruthClass.append(0)
    
    if clickbaitScores_test[i] > treshold:
        tetruthClass.append(1)
        count_high += 1
    else:
        tetruthClass.append(0)
        count_low += 1

print(count_high)
print(count_low)


mse = metrics.mean_squared_error(clickbaitScores_test,petruth_means)
print('Mean Squared Error = '+str(mse))

accuracy = metrics.accuracy_score(tetruthClass,petruthClass)
print('accuracy = '+str(accuracy))

precision = metrics.precision_score(tetruthClass,petruthClass)
print('precision_score = '+str(precision))

recall = metrics.recall_score(tetruthClass,petruthClass)
print('recall_score = '+str(recall))

f1 = metrics.f1_score(tetruthClass,petruthClass)
print('f1_score = '+str(f1))

conf = metrics.confusion_matrix(tetruthClass,petruthClass)
print(conf)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(32, dropout=0.2, recurrent_dropout=0.1)`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 3580 samples, validate on 398 samples
Epoch 1/2
3580/3580 [==============================] - 10s 3ms/step - loss: 0.0725 - val_loss: 0.1209
Epoch 2/2
  64/3580 [..............................] - ETA: 7s - loss: 0.0984

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `mse` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


3580/3580 [==============================] - 8s 2ms/step - loss: 0.0702 - val_loss: 0.0984
[[0.43323314]
 [0.41633886]
 [0.50286007]
 ...
 [0.44257727]
 [0.4293004 ]
 [0.41505843]]
624
2407
Mean Squared Error = 0.084177434
accuracy = 0.6004618937644342
precision_score = 0.27748294162244125
recall_score = 0.5865384615384616
f1_score = 0.3767370046320124
[[1454  953]
 [ 258  366]]


In [10]:
# train model postText_image_only
max_features = len(word2id.keys())
embedding_dims = EmbeddingSize
dropout_embedding = 0.2
maxlen = maxlen_posText

# build the keras LSTM model
model_postText = Sequential()

model_postText.add(Embedding(input_dim = max_features,
                    output_dim = embedding_dims,
                    weights = [embedding_matrix],
                    input_length = maxlen
                    ,trainable = False))
model_postText.add(Dropout(dropout_embedding))

model_postText.add(Bidirectional(GRU(256, dropout_W=0.2, dropout_U=0.5)))  
model_postText.add(Dense(1))
model_postText.add(Activation('sigmoid'))

model_postText.compile(loss='mse',
              optimizer='rmsprop')

batch_size = 64

earlystop_cb = keras.callbacks.EarlyStopping(monitor='mse', patience=7, verbose=1, mode='auto')

model_postText.fit(postTexts_train_emb_eqsub, clickbaitScores_train_eqsub, batch_size=batch_size, nb_epoch=2,
          validation_split=0.1, callbacks=[earlystop_cb])

# save the model
# model_postText.save('model_postText.h5')

# only save the weights
# model_postText.save_weights('weights_postText.hdf5')

petruth_means = model_postText.predict(postTexts_test_emb)
print(petruth_means)
tetruthClass = []
petruthClass = []

count_high = 0
count_low = 0
for i in range(len(clickbaitScores_test)):
  
    if petruth_means[i] > 0.5:
        petruthClass.append(1)
    else:
        petruthClass.append(0)
    
    if clickbaitScores_test[i] > 0.5:
        tetruthClass.append(1)
        count_high += 1
    else:
        tetruthClass.append(0)
        count_low += 1

print(count_high)
print(count_low)


mse = metrics.mean_squared_error(clickbaitScores_test,petruth_means)
print('Mean Squared Error = '+str(mse))

accuracy = metrics.accuracy_score(tetruthClass,petruthClass)
print('accuracy = '+str(accuracy))

precision = metrics.precision_score(tetruthClass,petruthClass)
print('precision_score = '+str(precision))

recall = metrics.recall_score(tetruthClass,petruthClass)
print('recall_score = '+str(recall))

f1 = metrics.f1_score(tetruthClass,petruthClass)
print('f1_score = '+str(f1))

conf = metrics.confusion_matrix(tetruthClass,petruthClass)
print(conf)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(256, dropout=0.2, recurrent_dropout=0.5)`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 3580 samples, validate on 398 samples
Epoch 1/2
3580/3580 [==============================] - 9s 2ms/step - loss: 0.0569 - val_loss: 0.0798
Epoch 2/2
 128/3580 [>.............................] - ETA: 5s - loss: 0.0453

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `mse` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


3580/3580 [==============================] - 7s 2ms/step - loss: 0.0490 - val_loss: 0.0398
[[0.5550343 ]
 [0.40422785]
 [0.71795154]
 ...
 [0.62577176]
 [0.15356608]
 [0.21472055]]
624
2407
Mean Squared Error = 0.040743686
accuracy = 0.8185417354008578
precision_score = 0.5544117647058824
recall_score = 0.6041666666666666
f1_score = 0.5782208588957055
[[2104  303]
 [ 247  377]]


In [11]:
# from keras.layers.merge import concatenate
from keras.layers import Concatenate
from keras.models import Model

# models merged
# postText
model_postText = Sequential()

model_postText.add(Embedding(input_dim = max_features,
                    output_dim = embedding_dims,
                    weights = [embedding_matrix],
                    input_length = maxlen
                    ,trainable = False))
model_postText.add(Dropout(dropout_embedding))

model_postText.add(Bidirectional(GRU(256, dropout_W=0.2, dropout_U=0.1)))
# model_postText.add(Dense(1))

# captions
model_captions = Sequential()

model_captions.add(Embedding(input_dim = max_features,
                    output_dim = embedding_dims,
                    weights = [embedding_matrix],
                    input_length = maxlen
                    ,trainable = False))
model_captions.add(Dropout(dropout_embedding))

model_captions.add(Bidirectional(GRU(256, dropout_W=0.2, dropout_U=0.1)))
# model_captions.add(Dense(1))

# combined
model_cap_post = Sequential()
# of course you must provide the input to result with will be your x3
model_cap_post.add(Dense(128))
model_cap_post.add(Dense(1, activation='sigmoid'))

merge_one = Concatenate([model_postText, model_captions])
final_merge = Concatenate([merge_one, model_cap_post])

# model_combined = Sequential()
# model_combined.add(concatenate(model_postText, model_captions))
# model_combined.add(Dense(1))
# model_combined.add(Activation('Sigmoid'))

                
model = Model(final_merge)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit([postTexts_train_emb_eqsub], [imageCaptions_train_emb_eqsub], 
                clickbaitScores_train_eqsub, batch_size=36, nb_epoch=20, 
                validation_split=0.1)

petruth_means = model_combined.predict(postTexts_test_emb)
print(petruth_means)
tetruthClass = []
petruthClass = []

count_high = 0
count_low = 0
for i in range(len(clickbaitScores_test)):

    if petruth_means[i] > 0.5:
        petruthClass.append(1)
    else:
        petruthClass.append(0)

    if clickbaitScores_test[i] > 0.5:
        tetruthClass.append(1)
        count_high += 1
    else:
        tetruthClass.append(0)
        count_low += 1

print(count_high)
print(count_low)


mse = metrics.mean_squared_error(clickbaitScores_test,petruth_means)
print('Mean Squared Error = '+str(mse))

accuracy = metrics.accuracy_score(tetruthClass,petruthClass)
print('accuracy = '+str(accuracy))

precision = metrics.precision_score(tetruthClass,petruthClass)
print('precision_score = '+str(precision))

recall = metrics.recall_score(tetruthClass,petruthClass)
print('recall_score = '+str(recall))

f1 = metrics.f1_score(tetruthClass,petruthClass)
print('f1_score = '+str(f1))

conf = metrics.confusion_matrix(tetruthClass,petruthClass)
print(conf)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(256, dropout=0.2, recurrent_dropout=0.1)`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(256, dropout=0.2, recurrent_dropout=0.1)`


TypeError: ignored

In [0]:
def plot_model(model):# Plot training & validation accuracy values
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

In [17]:
from keras.layers import concatenate
from keras.models import Model

# models merged
# postText
model_postText = Sequential()

model_postText.add(Embedding(input_dim = max_features,
                    output_dim = embedding_dims,
                    weights = [embedding_matrix],
                    input_length = maxlen_posText
                    ,trainable = False))
model_postText.add(Dropout(dropout_embedding))

model_postText.add(Bidirectional(GRU(256, dropout_W=0.2, dropout_U=0.1)))

# captions
model_captions = Sequential()

model_captions.add(Embedding(input_dim = max_features,
                    output_dim = embedding_dims,
                    weights = [embedding_matrix],
                    input_length = maxlen_captions
                    ,trainable = False))
model_captions.add(Dropout(dropout_embedding))

model_captions.add(Bidirectional(GRU(256, dropout_W=0.2, dropout_U=0.1)))


model_concat = concatenate([model_postText.output, model_captions.output], axis=-1)
model_concat = Dense(64, activation='relu')(model_concat)
model_concat = Dense(1, activation='sigmoid')(model_concat)
model_combined = Model(inputs=[model_postText.input, model_captions.input], outputs=model_concat)

model_combined.compile(loss='mse', optimizer='rmsprop')

model_combined.fit([postTexts_train_emb_eqsub, imageCaptions_train_emb_eqsub], 
                clickbaitScores_train_eqsub, batch_size=64, nb_epoch=20, validation_split=0.1)


petruth_means = model_combined.predict([postTexts_test_emb, imageCaptions_test_emb])
tetruthClass = []
petruthClass = []

count_high = 0
count_low = 0
for i in range(len(clickbaitScores_test)):

    if petruth_means[i] > 0.5:
        petruthClass.append(1)
    else:
        petruthClass.append(0)

    if clickbaitScores_test[i] > 0.5:
        tetruthClass.append(1)
        count_high += 1
    else:
        tetruthClass.append(0)
        count_low += 1

mse = metrics.mean_squared_error(clickbaitScores_test,petruth_means)
print('Mean Squared Error = '+str(mse))

accuracy = metrics.accuracy_score(tetruthClass,petruthClass)
print('accuracy = '+str(accuracy))

precision = metrics.precision_score(tetruthClass,petruthClass)
print('precision_score = '+str(precision))

recall = metrics.recall_score(tetruthClass,petruthClass)
print('recall_score = '+str(recall))

f1 = metrics.f1_score(tetruthClass,petruthClass)
print('f1_score = '+str(f1))

conf = metrics.confusion_matrix(tetruthClass,petruthClass)
print(conf)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(256, dropout=0.2, recurrent_dropout=0.1)`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(256, dropout=0.2, recurrent_dropout=0.1)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 3580 samples, validate on 398 samples
Epoch 1/20
3580/3580 [==============================] - 16s 5ms/step - loss: 0.0594 - val_loss: 0.0351
Epoch 2/20
3580/3580 [==============================] - 11s 3ms/step - loss: 0.0479 - val_loss: 0.0459
Epoch 3/20
3580/3580 [==============================] - 11s 3ms/step - loss: 0.0445 - val_loss: 0.0608
Epoch 4/20
3580/3580 [==============================] - 11s 3ms/step - loss: 0.0423 - val_loss: 0.0395
Epoch 5/20
3580/3580 [==============================] - 11s 3ms/step - loss: 0.0408 - val_loss: 0.0506
Epoch 6/20
3580/3580 [==============================] - 11s 3ms/step - loss: 0.0394 - val_loss: 0.0438
Epoch 7/20
3580/3580 [==============================] - 10s 3ms/step - loss: 0.0374 - val_loss: 0.0663
Epoch 8/20
3580/3580 [==============================] - 11s 3ms/step - loss: 0.0360 - val_loss: 0.0372
Epoch 9/20
3580/3580 [==============================] - 10s 3ms/step - loss: 0.0340 - val_loss: 0.0356
Epoch 10/20
3580/3580 [===